In [5]:
#Reset all variables during rerun
from IPython import get_ipython
get_ipython().magic('reset -sf')

normal='G:/My Drive/24-678 Computer Vision for Engineers/Project/normal/*.*'              ## Change path
osteoporosis='G:/My Drive/24-678 Computer Vision for Engineers/Project/osteoporosis/*.*'

In [6]:
import numpy as np
import cv2
import PIL
import glob
import matplotlib.pyplot as plt

In [7]:
def Image_block(path):
  file_list=glob.glob(path)
  original_files=[]   # Unchanged Lists
  my_list=[]          # Colored arrays stored after resizing with aspect ratio intact       
  gray_scale_list=[]  # List of gray_Scaled Images
  PCA_block=[]        # Block of Gray Scale can be directly used for PCA
  PCA_block_pad=[]    # Paddable block. Use above Dataset and custom DataLoader to process
  labels=[]           # Detect Labels Based on condition, Condition derived by Folder
  dimensions=[]

  d=path.split('/')
  s=d[5]
  #print(s)
  
  count=0
  for file in file_list:

    img=cv2.imread(file)

    (h,w)=np.shape(img)[:2]

    w1=w//8
    h1=h//8

    img_resized=cv2.resize(img,(w1,h1),interpolation=cv2.INTER_AREA)

    dimensions.append((w1,h1))        # Weight and height stored
    flat_img=img_resized.reshape(-1,3)
    #print(np.shape(flat_img))
    original_files.append(img)  # Deprecated Object
    my_list.append(flat_img)

    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray_scaled=cv2.resize(gray,(512,512))
    gray_resized=cv2.resize(gray,(w1,h1),interpolation=cv2.INTER_AREA)

    gray_scale_list.append(gray_scaled)

    flattened_img=gray_resized.flatten()

    PCA_block.append(flattened_img)
    PCA_block_pad.append(gray_scaled.flatten())

    if s=='normal':
      labels.append(0)
    else:
      labels.append(1)
    
    count +=1
  PCA_block=np.asarray(PCA_block)
  PCA_block_pad=np.asarray(PCA_block_pad)
  print(np.shape(PCA_block))
  print(np.shape(PCA_block_pad))
  print(count)

  return original_files,my_list,gray_scale_list,PCA_block,PCA_block_pad,labels,dimensions

In [8]:
a,b,c,d,e,f,g=Image_block(normal)

(186,)
(186, 262144)
186


C:\Users\jaydu\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [9]:
h,i,j,k,l,m,n = Image_block(osteoporosis)

(186,)
(186, 262144)
186


In [10]:
def pseudo(name,image):
    img2 = image.copy().astype("float32")
    for i in range(img2.shape[2]):
        img2[:,:,i] = (img2[:,:,i]-np.min(img2[:,:,i]))/(np.max(img2[:,:,i])-np.min(img2[:,:,i]))
        img2[:,:,i] *= 255

    lut_list_bgr = np.zeros((256,1,3)).astype("uint8")

    for i in range(128,192):
        lut_list_bgr[i,0,2] = 4*(i-127.5)
        lut_list_bgr[i,0,1] = 255

    for j in range(192,256):
        lut_list_bgr[j,0,2] = 255
        lut_list_bgr[j,0,1] = 4*(191.25-j) + 255

    for k in range(0,64):
        lut_list_bgr[k,0,0] = 255
        lut_list_bgr[k,0,1] = 4*k

    for m in range(64,128):
        lut_list_bgr[m,0,0] = 4*(63.75-m)+255
        lut_list_bgr[m,0,1] = 255

    lut_list_bgrop = lut_list_bgr.astype("uint8")
    img2 = img2.astype("uint8")

    imp = cv2.LUT(img2,lut_list_bgrop)
    
    img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    
    centre_coord = np.argwhere(img_gray==np.amax(img_gray))
    centre = (np.mean(centre_coord,axis=0)).astype(int)
    count = 0
    for i in centre_coord:
        imp = cv2.circle(imp,(i[1],i[0]),15,(255,255,255),2)
        count += 1
#     imp = cv2.line(imp,(centre[1]-25,centre[0]),(centre[1]+25,centre[0]),(255,255,255),2)
#     imp = cv2.line(imp,(centre[1],centre[0]-25),(centre[1],centre[0]+25),(255,255,255),2)
    
#     cv2.imshow("applyColorMap",imp)
#     cv2.waitKey(0)
#     save = input("Save the image? ")#(press y to save image)
#     if save == "Y" or save == "y":
    cv2.imwrite("OG"+str(name)+"_pseudo.png",imp)
    #cv2.waitKey(0)
    return count, imp

In [11]:
# for elem in range(186):
#     pseudo("Normal_"+str(elem),a[elem])

In [ ]:
for elem in range(186):
    pseudo("Osteoporotic_"+str(elem),h[elem])